## Checking the performance of quality metric from hembedder 

In [2]:
from setup_cc import *
import cProfile
import umap
import matplotlib.pyplot as plt
import seaborn as sns
import cProfile, pstats, io
from pstats import SortKey

sample_index = cell_dyn.sample(40000).index
sampled_cd = np.asarray(cell_dyn.loc[sample_index], dtype=np.float32)
#sampled_embedded = np.asarray(dm6.loc[sample_index],dtype=np.float16)





In [3]:
#trustworthiness --> sklearn
# distance correlation correlation --> zelf chefffen
# knn-overlap - distance curve and integral
# poincarre


### knn_overlap -> find knn in true space random sample. Then embedding do the same for indices -> jaccard score
__author__ = "Bram van ES", "Huibert-Jan Joosse", "Chontira Chumsaeng"

from scipy.spatial.distance import jaccard, hamming
from scipy.spatial.distance import pdist
import scipy as sc

import numpy as np
from numpy.random import default_rng
from sklearn.manifold import trustworthiness
import dcor
import faiss
import time
import os
from tabulate import tabulate
from collections import defaultdict
from sklearn.neighbors import NearestNeighbors
from joblib import Parallel, delayed

import ctypes
from ctypes import cdll

import hembedder.utils._cpython._metrics_cy as metrics_cy

from functools import lru_cache
from numba import jit, njit

from tqdm import tqdm

from sklearn.metrics import pairwise_distances, pairwise_distances_chunked

"""
sources:
https://github.com/samueljackson92/coranking/tree/master/src
https://github.com/gdkrmr/coRanking/tree/master/R
"""

# source : https://timsainburg.com/coranking-matrix-python-numba.html
def compute_ranking_matrix_parallel(D):
    """Compute ranking matrix in parallel. Input (D) is distance matrix"""
    # if data is small, no need for parallel
    if len(D) > 1000:
        n_jobs = -1
    else:
        n_jobs = 1
    r1 = Parallel(n_jobs, prefer="threads")(
        delayed(np.argsort)(i)
        for i in tqdm(D.T, desc="computing rank matrix", leave=False)
    )
    r2 = Parallel(n_jobs, prefer="threads")(
        delayed(np.argsort)(i)
        for i in tqdm(r1, desc="computing rank matrix", leave=False)
    )
    # write as a single array
    r2_array = np.zeros((len(r2), len(r2[0])), dtype="int32")
    for i, r2row in enumerate(tqdm(r2, desc="concatenating rank matrix", leave=False)):
        r2_array[i] = r2row
    return r2_array


@njit(fastmath=True)
def populate_Q(Q, i, m, R1, R2):
    """populate coranking matrix using numba for speed"""
    for j in range(m):
        k = R1[i, j]
        l = R2[i, j]
        Q[k, l] += 1
    return Q


def iterate_compute_distances(data):
    """Compute pairwise distance matrix iteratively, so we can see progress"""
    n = len(data)
    D = np.zeros((n, n), dtype="float32")
    col = 0
    for i, distances in enumerate(
        pairwise_distances_chunked(data, n_jobs=-1),
    ):
        D[col : col + len(distances)] = distances
    return D


def compute_coranking_matrix(data_ld, data_hd=None, D_hd=None, leave = True):
    """Compute the full coranking matrix"""

    # compute pairwise probabilities
    if D_hd is None:
        D_hd = iterate_compute_distances(data_hd)

    D_ld = iterate_compute_distances(data_ld)
    n = len(D_ld)
    # compute the ranking matrix for high and low D
    rm_hd = compute_ranking_matrix_parallel(D_hd)
    rm_ld = compute_ranking_matrix_parallel(D_ld)

    # compute coranking matrix from_ranking matrix
    m = len(rm_hd)
    Q = np.zeros(rm_hd.shape, dtype="int16")
    for i in tqdm(range(m), desc="computing coranking matrix", leave = leave):
        Q = populate_Q(Q, i, m, rm_hd, rm_ld)

    Q = Q[1:, 1:]
    return Q


@njit(fastmath=True)
def qnx_crm(Q: np.array, n_neighbours: int):
    """Average Normalized Agreement Between K-ary Neighborhoods (QNX)
    # QNX measures the degree to which an embedding preserves the local
    # neighborhood around each observation. For a value of K, the K closest
    # neighbors of each observation are retrieved in the input and output space.
    # For each observation, the number of shared neighbors can vary between 0
    # and K. QNX is simply the average value of the number of shared neighbors,
    # normalized by K, so that if the neighborhoods are perfectly preserved, QNX
    # is 1, and if there is no neighborhood preservation, QNX is 0.
    #
    # For a random embedding, the expected value of QNX is approximately
    # K / (N - 1) where N is the number of observations. Using RNX
    # (\code{rnx_crm}) removes this dependency on K and the number of
    # observations.
    #
    # @param crm Co-ranking matrix. Create from a pair of distance matrices with
    # \code{coranking_matrix}.
    # @param k Neighborhood size.
    # @return QNX for \code{k}.
    # @references
    # Lee, J. A., & Verleysen, M. (2009).
    # Quality assessment of dimensionality reduction: Rank-based criteria.
    # \emph{Neurocomputing}, \emph{72(7)}, 1431-1443.

    Python reimplmentation of code by jlmelville
    (https://github.com/jlmelville/quadra/blob/master/R/neighbor.R)
    source: https://timsainburg.com/coranking-matrix-python-numba.html
    """
    qnx_crm_sum = np.sum(Q[:n_neighbours, :n_neighbours])
    return qnx_crm_sum / (n_neighbours * len(Q))


@njit(fastmath=True)
def rnx_crm(Q: np.array, n_neighbours: int):
    """Rescaled Agreement Between K-ary Neighborhoods (RNX)
    # RNX is a scaled version of QNX which measures the agreement between two
    # embeddings in terms of the shared number of k-nearest neighbors for each
    # observation. RNX gives a value of 1 if the neighbors are all preserved
    # perfectly and a value of 0 for a random embedding.
    #
    # @param crm Co-ranking matrix. Create from a pair of distance matrices with
    # \code{coranking_matrix}.
    # @param k Neighborhood size.
    # @return RNX for \code{k}.
    # @references
    # Lee, J. A., Renard, E., Bernard, G., Dupont, P., & Verleysen, M. (2013).
    # Type 1 and 2 mixtures of Kullback-Leibler divergences as cost functions in
    # dimensionality reduction based on similarity preservation.
    # \emph{Neurocomputing}, \emph{112}, 92-108.

    Python reimplmentation of code by jlmelville
    (https://github.com/jlmelville/quadra/blob/master/R/neighbor.R)
    source: https://timsainburg.com/coranking-matrix-python-numba.html
    """
    n = len(Q)
    return ((qnx_crm(Q, n_neighbours) * (n - 1)) - n_neighbours) / (
        n - 1 - n_neighbours
    )


@njit(fastmath=True)
def rnx_auc_crm(Q: np.array):
    """Area Under the RNX Curve
    # The RNX curve is formed by calculating the \code{rnx_crm} metric for
    # different sizes of neighborhood. Each value of RNX is scaled according to
    # the natural log of the neighborhood size, to give a higher weight to smaller
    # neighborhoods. An AUC of 1 indicates perfect neighborhood preservation, an
    # AUC of 0 is due to random results.
    #
    # param crm Co-ranking matrix.
    # return Area under the curve.
    # references
    # Lee, J. A., Peluffo-Ordo'nez, D. H., & Verleysen, M. (2015).
    # Multi-scale similarities in stochastic neighbour embedding: Reducing
    # dimensionality while preserving both local and global structure.
    # \emph{Neurocomputing}, \emph{169}, 246-261.

    Python reimplmentation of code by jlmelville
    (https://github.com/jlmelville/quadra/blob/master/R/neighbor.R)
    source: https://timsainburg.com/coranking-matrix-python-numba.html
    """
    n = len(Q)
    num = 0
    den = 0

    qnx_crm_sum = 0
    for k in range(1, n - 2):
        # for k in (range(1, n - 2)):
        qnx_crm_sum += (
            np.sum(Q[(k - 1), :k]) + np.sum(Q[:k, (k - 1)]) - Q[(k - 1), (k - 1)]
        )
        qnx_crm = qnx_crm_sum / (k * len(Q))
        rnx_crm = ((qnx_crm * (n - 1)) - k) / (n - 1 - k)
        num += rnx_crm / k
        den += 1 / k
    return num / den


class CDEmbeddingPerformance:
    """
    Class for calulating the embedding quality. Metrics include trustworthiness,
    Knn overlap, distance correlation, and random triplet scores
    """

    def __init__(
        self,
        metric="euclidean",
        n_neighbours: int = 50,
        knn_dist: str = "jaccard",
        dcor_level: int = 2,
        num_triplets: int = 5,
        dtype=np.float32,
        scaled: bool = False,
    ):
        """
        Setting up parameters for the quality metircs

        Paramters
        ---------
        metric: string or function
            distance metric for trustworiness and distance correlation
        n_neighbours:int
            number of neighbours for trustworiness and knn overlap scores
        knn_dist:string
            distance metric for calculating overlap between neighbours in knn overlap.
            'hamming' or 'jaccard'
        dcor_level:int
            depth of correlation; 1 is distance correlation of data, 2 is distance
            correlation of distances
        num_triplets:int
            paramter for random triplets calculation.
        dtype: float
            for setting type of np.array to work with. If the data set is
            too large use np.float16
        scaled: bool
            whether to use a scaled version of Qnx
        """
        self.metric = metric
        self.n_neighbours = n_neighbours
        self.knn_dist = knn_dist
        self.num_triplets = num_triplets
        self.dtype = dtype
        self.dcor_level = dcor_level
        self.scaled = scaled

    def _get_coranking_matrix(self, X_org: np.array, X_emb: np.array, backend="numba"):
        """
        Function for returning the coranking matrix of the original data and the embedded data

        Parameters
        ----------
        X_org:np.array
            the original dataset as np.array
        X_emb:np.array
            the embedded data as np.array

        Returns
        -----------
        coranking matrix as np.array
        """

        if backend == "python":
            # source: https://github.com/samueljackson92/coranking/blob/master/coranking/_coranking.py
            n, m = X_org.shape
            high_distance = sc.spatial.distance.squareform(
                sc.spatial.distance.pdist(X_org)
            )
            low_distance = sc.spatial.distance.squareform(
                sc.spatial.distance.pdist(X_emb)
            )

            high_ranking = high_distance.argsort(axis=1).argsort(axis=1)
            low_ranking = low_distance.argsort(axis=1).argsort(axis=1)

            Q, xedges, yedges = np.histogram2d(
                high_ranking.flatten(), low_ranking.flatten(), bins=n
            )
            Q = Q.astype(np.int32)
            return Q[:1, :1]
        elif backend == "numba":
            Q = compute_coranking_matrix(data_ld=X_emb, data_hd=X_org).astype(np.int32)
            return Q
        elif backend == "cython":
            Q = metrics_cy.Qmatrix(X_org, X_emb)
            return Q
        elif backend == "ctype":
            script_path = "/".join(str(os.path.abspath(__file__)).split("/")[:-1])
            coranking = cdll.LoadLibrary(
                os.path.join(script_path, "_ctypes/coranking.so")
            )
            # coranking = ctypes.CDLL(os.path.join(script_path, "_ctypes/coranking.so"))
            # from hembedder.utils._ctypes import coranking

            # print("Making ravelled arrays")
            # X_org_vector = np.ravel(X_org, order="F")
            # X_emb_vector = np.ravel(X_emb, order="F")

            print("Making C-types variables")
            c_double_C = ctypes.POINTER(ctypes.c_double)
            X_org_C = X_org.ctypes.data_as(c_double_C)
            X_emb_C = X_emb.ctypes.data_as(c_double_C)

            # print("Setting up C-types function argument and return types")
            coranking.euclidean.restype = None
            coranking.euclidean.argtypes = [
                ctypes.POINTER(ctypes.c_double),
                ctypes.c_int,
                ctypes.c_int,
                ctypes.POINTER(ctypes.c_double),
            ]

            print("Calling Ctype function on original data")
            N = X_org.shape[0]
            Dor = X_org.shape[1]

            DD = np.zeros((N, N), dtype=self.dtype)
            DD_ptr = DD.ctypes.data_as(ctypes.POINTER(ctypes.c_double))
            coranking.euclidean(X_org_C, N, Dor, DD_ptr)
            high_distance = DD.copy()

            print("Calling Ctype function on embedded data")
            Demb = X_emb.shape[1]

            DD = np.zeros((N, N), dtype=self.dtype)
            DD_ptr = DD.ctypes.data_as(ctypes.POINTER(ctypes.c_double))
            coranking.euclidean(X_emb_C, N, Demb, DD_ptr)
            low_distance = DD.copy()

            print("Calling Ctype function on rankmatrix, original")
            coranking.rankmatrix.restype = None
            coranking.rankmatrix.argtypes = [
                ctypes.POINTER(ctypes.c_double),
                ctypes.c_int,
                ctypes.POINTER(ctypes.c_int),
            ]

            Rm = np.zeros((N, N), dtype=np.int32)
            Rm_ptr = Rm.ctypes.data_as(ctypes.POINTER(ctypes.c_int))
            coranking.rankmatrix(
                high_distance.ctypes.data_as(ctypes.POINTER(ctypes.c_double)),
                N,
                Rm_ptr,
            )
            high_ranking = Rm.copy()

            print("Calling Ctype function on rankmatrix, embedded")
            Rm = np.zeros((N, N), dtype=np.int32)
            Rm_ptr = Rm.ctypes.data_as(ctypes.POINTER(ctypes.c_int))
            coranking.rankmatrix(
                low_distance.ctypes.data_as(ctypes.POINTER(ctypes.c_double)),
                N,
                Rm_ptr,
            )
            low_ranking = Rm.copy()

            ##
            coranking.coranking.restype = None
            coranking.coranking.argtypes = [
                ctypes.POINTER(ctypes.c_int),
                ctypes.POINTER(ctypes.c_int),
                ctypes.c_int,
                ctypes.POINTER(ctypes.c_int),
            ]

            print("Calling Ctype function on Qmatrix")
            Q = np.zeros((N - 1, N - 1), dtype=np.int32)
            Q_ptr = Q.ctypes.data_as(ctypes.POINTER(ctypes.c_int))
            coranking.coranking(
                high_ranking.ctypes.data_as(ctypes.POINTER(ctypes.c_int)),
                low_ranking.ctypes.data_as(ctypes.POINTER(ctypes.c_int)),
                N,
                Q_ptr,
            )
            return Q
        else:
            raise ValueError("Backend not supported")

    def _return_Qnx(self, X_org: np.array, X_emb: np.array, Q: np.array = None):
        """
        Function for returning Qnx score from the original data and the embedded data

        Parameters
        ----------
        X_org:np.array
            the original dataset as np.array
        X_emb:np.array
            the embedded data as np.array
        Q:np.array
            the coranking matrix as np.array

        Returns
        -----------
        Qnx score as float
        """
        if Q is None:
            Q = self._get_coranking_matrix(X_org, X_emb)
        return metrics_cy.Qnx(Q, self.n_neighbours, self.scaled)

    def _return_Qtrustworthiness(
        self, X_org: np.array, X_emb: np.array, Q: np.array = None
    ):
        """
        Function for returning trustworthiness score from sklearn.manifold.

        Parameters
        ----------
        X_org:np.array
            the original dataset as np.array
        X_emb:np.array
            the embedded data as np.array
        Q:np.array
            the coranking matrix as np.array

        Returns
        -----------
        Trustworthiness score between 0 and 1. Higher means better
        """
        if Q is None:
            Q = self._get_coranking_matrix(X_org, X_emb)
        return metrics_cy.trustworthiness(Q, self.n_neighbours)

    def _return_Qcontinuity(self, X_org: np.array, X_emb: np.array, Q: np.array = None):
        """
        Function for returning trustworthiness score from sklearn.manifold.

        Parameters
        ----------
        X_org:np.array
            the original dataset as np.array
        X_emb:np.array
            the embedded data as np.array
        Q:np.array
            the coranking matrix as np.array

        Returns
        -----------
        Continuity score between 0 and 1. Higher means better
        """
        if Q is None:
            Q = self._get_coranking_matrix(X_org, X_emb)
        return metrics_cy.continuity(Q, self.n_neighbours)

    def _return_LCMC(self, X_org: np.array, X_emb: np.array, Q: np.array = None):
        """
        Function for returning LCMC score from the original data and the embedded data

        Parameters
        ----------
        X_org:np.array
            the original dataset as np.array
        X_emb:np.array
            the embedded data as np.array
        Q:np.array
            the coranking matrix as np.array

        Returns
        -----------
        LCMC score as float
        """
        if Q is None:
            Q = self._get_coranking_matrix(X_org, X_emb)
        return metrics_cy.LCMC(Q, self.n_neighbours)

    def _return_nMRRE(self, X_org: np.array, X_emb: np.array, Q: np.array = None):
        """
        Function for returning LCMC score from the original data and the embedded data

        Parameters
        ----------
        X_org:np.array
            the original dataset as np.array
        X_emb:np.array
            the embedded data as np.array
        Q:np.array
            the coranking matrix as np.array

        Returns
        -----------
        nMRRE score as float
        """
        if Q is None:
            Q = self._get_coranking_matrix(X_org, X_emb)
        return metrics_cy.nMRRE(Q, self.n_neighbours)

    def _return_vMRRE(self, X_org: np.array, X_emb: np.array, Q: np.array = None):
        """
        Function for returning LCMC score from the original data and the embedded data

        Parameters
        ----------
        X_org:np.array
            the original dataset as np.array
        X_emb:np.array
            the embedded data as np.array
        Q:np.array
            the coranking matrix as np.array

        Returns
        -----------
        vMRRE score as float
        """
        if Q is None:
            Q = self._get_coranking_matrix(X_org, X_emb)
        return metrics_cy.vMRRE(Q, self.n_neighbours)

    def _return_qnx_crm(self, X_org: np.array, X_emb: np.array,Q: np.array=None):
        """
        Function for returning qnx_crm score from the coranking matrix

        Parameters
        ----------
        X_org:np.array
            the original dataset as np.array
        X_emb:np.array
            the embedded data as np.array
        Q:np.array
            the coranking matrix as np.array

        Returns
        -----------
        qnx_crm score as float
        """
        if Q is None:
            Q = self._get_coranking_matrix(X_org, X_emb)
        return qnx_crm(Q, self.n_neighbours)

    def _return_rnx_crm(self, X_org: np.array, X_emb: np.array, Q: np.array=None):
        """
        Function for returning rnx_crm score from the coranking matrix

        Parameters
        ----------
       X_org:np.array
            the original dataset as np.array
        X_emb:np.array
            the embedded data as np.array
        Q:np.array
            the coranking matrix as np.array

        Returns
        -----------
        rnx_crm score as float
        """
        if Q is None:
            Q = self._get_coranking_matrix(X_org, X_emb)
        return rnx_crm(Q, self.n_neighbours)

    def _return_rnx_auc_crm(self, X_org: np.array, X_emb: np.array, Q: np.array=None):
        """
        Function for returning rnx_auc_crm score from the coranking matrix

        Parameters
        ----------
        X_org:np.array
            the original dataset as np.array
        X_emb:np.array
            the embedded data as np.array
        Q:np.array
            the coranking matrix as np.array

        Returns
        -----------
        rnx_auc_crm score as float
        """
        if Q is None:
            Q = self._get_coranking_matrix(X_org, X_emb)
        return rnx_auc_crm(Q)

    def _return_trustworthiness(self, X_org: np.array, X_emb: np.array):
        """
        Function for returning trustworthiness score from sklearn.manifold.

        Parameters
        ----------
        X_org:np.array
            the original dataset as np.array
        X_emb:np.array
            the embedded data as np.array

        Returns
        -----------
        Trustworithiness score between 0 and 1. Higher means better
        """
        return trustworthiness(
            X_org, X_emb, n_neighbors=self.n_neighbours, metric=self.metric
        )

    @staticmethod
    def _create_knn_search(X, k, dtype):
        """
        HELPER function for knn_overlap
        """
        index = faiss.IndexFlatL2(X.shape[1])
        index.add(X.astype(dtype))
        D, I = index.search(X.astype(dtype), k)
        return D, I

    def _return_knn_overlap(
        self, X_org: np.array, X_emb: np.array, knn_return_median: bool = True
    ):
        """
        Function for returning nearest neighbourhood overlap score.
        Overlap between the high dimension and low dimension data

        Parameters
        ----------
        X_org:np.array
            the original dataset as np.array
        X_emb:np.array
            the embedded data as np.array
        knn_return_median:bool
            whether to return the median of the knn overlap scores.
            This should be true if knn is to be used with other methods here.

        Returns
        -----------
        knn overlap score between 0 and 1. Lower means better
        """
        D, I = self._create_knn_search(X_org, self.n_neighbours, self.dtype)
        D_emb, I_emb = self._create_knn_search(X_emb, self.n_neighbours, self.dtype)
        ds_arry = np.zeros(I.shape[0])
        dist = None
        if self.knn_dist == "jaccard":
            dist = jaccard
        elif self.knn_dist == "hamming":
            dist = hamming
        else:
            raise Exception(
                f"{self.knn_dist} is not a recognised distance"
                + " measure for KNN overlap. Please use 'jaccard' or 'hamming'"
            )
        for i in range(I.shape[0]):
            ds_arry[i] = dist(I[i, :], I_emb[i, :])
        return np.median(ds_arry) if knn_return_median else ds_arry

    def _return_distance_correlation(self, X_org: np.array, X_emb: np.array):
        """
        Function for returning distance correlation from dcor between the
        high dimension and low dimension data

        Parameters
        ----------
        X_org:np.array
            the original dataset as np.array
        X_emb:np.array
            the embedded data as np.array

        Returns
        -----------
        distance correlation score between 0 and 1. Higher means better
        """
        if self.dcor_level == 1:
            return dcor.distance_correlation(X_org, X_emb)
        elif self.dcor_level == 2:
            return dcor.distance_correlation(
                pdist(X_org, metric="cityblock"), pdist(X_emb, metric="cityblock")
            )
        else:
            raise Exception(
                f"{self.dcor_level} is not a recognised level for"
                + " distance correlation. Please use 1 or 2"
            )

    def _return_dynamic_distance_correlation(
        self, X_org: np.array, X_emb: np.array, n_bins=5
    ):
        """
        Function for returning dynamic distance correlation from dcor between the
        high dimension and low dimension data

        Parameters
        ----------
        X_org:np.array
            the original dataset as np.array
        X_emb:np.array
            the embedded data as np.array
        n_bins: int
            the number of quantile bins to extract the distance-distance correlation
        Returns
        -----------
        dynamic distance correlation score between 0 and 1. Higher means better
        """

        dist_org = pdist(X_org, metric="cityblock")
        dist_emb = pdist(X_emb, metric="cityblock")

        d_min = min(dist_org)
        d_max = max(dist_org)
        d_bins = np.quantile(dist_org, np.linspace(0, 1, n_bins + 1))
        d_ranges = list(zip(d_bins[:-1], d_bins[1:]))
        res = []
        for d_range in d_ranges:
            idx = (dist_org >= d_range[0]) & (dist_org <= d_range[1])
            res.append(dcor.distance_correlation(dist_emb[idx], dist_org[idx]))
        return res

    @staticmethod
    def calculate_distance_random_triplets(X: np.array, anchors, triplets):
        """
        HELPER function for random_triplet_eval to calculate distance
        for X and generate the labels
        """
        b = np.broadcast(anchors, triplets)
        distances = np.empty(b.shape)
        distances.flat = [np.linalg.norm(X[u] - X[v]) for (u, v) in b]
        labels = distances[:, :, 0] < distances[:, :, 1]
        return labels

    @staticmethod
    def calculate_anchors_and_triplets(X: np.array, num_triplets: int):
        """
        HELPER function for random_triplet_eval to create the
        achors and triplets for evaluating thetriplets violation
        """
        anchors = np.arange(X.shape[0])
        rng = default_rng()
        triplets = rng.choice(anchors, (X.shape[0], num_triplets, 2))
        triplet_labels = np.zeros((X.shape[0], num_triplets))
        anchors = anchors.reshape((-1, 1, 1))
        return anchors, triplets

    def random_triplet_eval(self, X_org: np.array, X_emb: np.array):
        """
        Author: Haiyang Huang https://github.com/hyhuang00/scRNA-DR2020/blob/main/experiments/run_eval.py
        This is a function that is used to evaluate the lower dimension embedding.
        An triplet satisfaction score is calculated by evaluating how many randomly
        selected triplets have been violated. Each point will generate 5 triplets.

        Parameters
        ----------
            X_org: A numpy array with the shape [N, p]. The higher dimension embedding
            of some dataset. Expected to have some clusters.
            X_emb: A numpy array with the shape [N, k]. The lower dimension embedding
                of some dataset. Expected to have some clusters as well.

        Returns
        ----------
            acc: The score generated by the algorithm.
        """
        # Sampling Triplets
        # Five triplet per point
        anchors, triplets = self.calculate_anchors_and_triplets(
            X_org, self.num_triplets
        )

        # Calculate the distances and generate labels
        labels = self.calculate_distance_random_triplets(X_org, anchors, triplets)

        # Calculate distances for LD
        pred_vals = self.calculate_distance_random_triplets(X_emb, anchors, triplets)

        # Compare the labels and return the accuracy
        correct = np.sum(pred_vals == labels)
        acc = correct / X_org.shape[0] / self.num_triplets
        return acc

    def neighbor_kept_ratio_eval(self, X_org: np.array, X_emb: np.array):
        """
        Author: Haiyang Huang https://github.com/hyhuang00/scRNA-DR2020/blob/main/experiments/run_eval.py
        This is a function that evaluates the local structure preservation.
        A nearest neighbor set is constructed on both the high dimensional space and
        the low dimensional space.

        Input:
            X_org: A numpy array with the shape [N, p]. The higher dimension embedding
            of some dataset. Expected to have some clusters.
            X_emb: A numpy array with the shape [N, k]. The lower dimension embedding
                of some dataset. Expected to have some clusters as well.

        Output:
            acc: The score generated by the algorithm.
        """
        nn_hd = NearestNeighbors(n_neighbors=self.n_neighbours + 1)
        nn_ld = NearestNeighbors(n_neighbors=self.n_neighbours + 1)
        nn_hd.fit(X_org)
        nn_ld.fit(X_emb)
        # Construct a k-neighbors graph, where 1 indicates a neighbor relationship
        # and 0 means otherwise, resulting in a graph of the shape n * n
        graph_hd = nn_hd.kneighbors_graph(X_org).toarray()
        graph_hd -= np.eye(X_org.shape[0])  # Removing diagonal
        graph_ld = nn_ld.kneighbors_graph(X_emb).toarray()
        graph_ld -= np.eye(X_org.shape[0])  # Removing diagonal
        neighbor_kept = np.sum(graph_hd * graph_ld).astype(self.dtype)
        neighbor_kept_ratio = neighbor_kept / self.n_neighbours / X_org.shape[0]
        return neighbor_kept_ratio

    def score(
        self,
        X_org: np.array,
        X_emb: np.array,
        subsampling: int = 1000,
        num_iter: int = 10,
        return_results: bool = False,
    ):
        """
        Return embedding trustworithness, knn overlap, distance correlation,
        and random triplets scores.

        Parameters
        ----------
        X_org:np.array
            the original dataset as np.array
        X_emb:np.array
            the embedded data as np.array
        subsampling:int
            the number of samples for the data to be subsampled down to
        num_iter:
            the amount of iteration for the algorithms to cycle through
            for calculating the scores
        return_results:bool
            whether to return the results as dictionary

        Returns
        ----------
        final_results:dict, optional
            the calculated results in a dictionary.

        """
        evaluators = {
            "Trustworthiness": self._return_trustworthiness,
            "Knn overlap": self._return_knn_overlap,
            "Distance correlation": self._return_distance_correlation,
            "Random triplets": self.random_triplet_eval,
            "neighbor kept ratio": self.neighbor_kept_ratio_eval,
        }
        final_results = score_subsampling(
            X_org,
            X_emb,
            evaluators=evaluators,
            size=subsampling,
            num_iter=num_iter,
            verbose=True,
            return_dict=True,
        )
        if return_results:
            return final_results


def metrics_scores_iter(
    x: np.array,
    output: np.array,
    evaluators: dict,
    verbose: bool = True,
    return_dict: bool = False,
    **args
):
    """Calculates scores for embedder using different metrics (evaluators).

    Parameters
    ---------
    x: np.array
        original unemedded data
    output: np.array
        output array from the embedder for evaluation
    evaluators: dict
        further arguments to include the metrics (as function statement in a dict)
        if the functions take x and output as arguments.
    verbose: bool, optional
        whether to print the results.

    Returns
    ---------
    results: dict
        dictonary of metrics and their calculated scores.
    """
    results = {}
    for name, metric in evaluators.items():
        results.update({name: metric(x, output,**args)})
    if verbose:
        print_metric_scores(results)
    if return_dict:
        return results


def print_metric_scores(results: dict):
    """Print mertic scores in a table format.

    Parameters
    ---------
    results: dict
            dictionary of results to be printed in a table format.
    """
    tab = []
    for metric, score in results.items():
        tab.append([metric, score])
    print(tabulate(tab, headers=["Metric", "Score"]))
    print("\n")


def print_means_metric_scores(results: dict):
    """Print means of metric scores in a table format with standard deviation.

    Parameters
    ---------
    results: dict
            dictionary of results to be printed in a table format.
    """
    tab = []
    for metric, score in results.items():
        tab.append([metric, score[0][0], score[0][1]])
    print(tabulate(tab, headers=["Metric", "Mean", "Standard deviation"]))
    print("\n")


def get_results(args):
    """HELPER function for parallelisation of this function in score_subsampling"""
    sample = np.random.choice(np.arange(len(args["X"])), size=args["size"])
    X_org_sub = args["X"][sample, :]
    X_emb_sub = args["output"][sample, :]
    results = metrics_scores_iter(
        X_org_sub, X_emb_sub, args["evaluators"], verbose=False, return_dict=True
    )
    for name, score in results.items():
        args["results"][name].append(score)
    return args["results"]


def score_subsampling(
    X: np.array,
    output: np.array,
    evaluators: dict,
    size: int = 1000,
    num_iter: int = 10,
    verbose: bool = True,
    return_dict: bool = False,
):
    """Calculates quality metric scores with subsampling to reduce processing time.

    Parameters
    ---------
    x: np.array
        original unemedded data
    output: np.array
        output array from the embedder for evaluation
    evaluators: dict
        further arguments to include the metrics (as function statement in a dict)
        if the functions take x and output as arguments.
    size:int
        the number of samples for the data to be subsampled down to
    num_iter:
        the amount of iteration for the algorithms to cycle through for calculating the scores
    verbose: bool, optional
        whether to print the results.

    Returns
    ---------
    results: dict
        dictonary of metrics and their calculated scores.
    """
    results = defaultdict(list)
    method_start = time.time()
    parallel_param = {
        "X": X,
        "output": output,
        "evaluators": evaluators,
        "size": size,
        "results": results,
    }
    results = Parallel(n_jobs=8, verbose=False, pre_dispatch="1.5*n_jobs")(
        delayed(get_results)((parallel_param)) for _ in range(num_iter)
    )

    results_dict = defaultdict(list)
    for i in range(len(results)):
        for k, v in results[i].items():
            results_dict[k].append(v)

    final_results = defaultdict(list)
    for name, scores in results_dict.items():
        final_results[name].append([np.mean(scores), np.std(scores)])
    if verbose:
        print_means_metric_scores(final_results)
        print(f"Supsampling of {size} samples for {num_iter} rounds each")
        print(f"Time taken {round((time.time()-method_start)/60, 2)} minutes")
    if return_dict:
        return final_results


# add main
if __name__ == "__main__":
    # make synthetic data
    from sklearn.datasets import make_regression
    from sklearn.decomposition import PCA

    data_original, _ = make_regression(
        n_samples=500, n_features=100, n_informative=10, random_state=42
    )
    data_embedding = PCA(n_components=10).fit_transform(data_original)

    # make evaluator
    quality = CDEmbeddingPerformance()

    # make Qmatrix
    print("Making Qmatrix")
    Qmatrix = quality._get_coranking_matrix(
        X_org=data_original, X_emb=data_embedding, backend="numba"
    )
    print(f"Qmatrix: {Qmatrix[:10, :10]}")

    # make scores
    print("Making scores")

    Qnx = quality._return_Qnx(X_org=data_original, X_emb=data_embedding, Q=Qmatrix)
    print(f"Qnx: {Qnx}")

    Qtrust = quality._return_Qtrustworthiness(
        X_org=data_original, X_emb=data_embedding, Q=Qmatrix
    )
    print(f"Qtrust: {Qtrust}")

    Qcont = quality._return_Qcontinuity(
        X_org=data_original, X_emb=data_embedding, Q=Qmatrix
    )
    print(f"Qcont: {Qcont}")

    Qlcmc = quality._return_LCMC(X_org=data_original, X_emb=data_embedding, Q=Qmatrix)
    print(f"Qlcmc: {Qlcmc}")

    QnMRRE = quality._return_nMRRE(X_org=data_original, X_emb=data_embedding, Q=Qmatrix)
    print(f"QnMRRE: {QnMRRE}")

    QvMRRE = quality._return_vMRRE(X_org=data_original, X_emb=data_embedding, Q=Qmatrix)
    print(f"QvMRRE: {QvMRRE}")

    DistCorr = quality._return_dynamic_distance_correlation(
        X_org=data_original, X_emb=data_embedding
    )
    print(f"DistCorr: {DistCorr}")

    print(
        f"DistCorr level 1: {dcor.distance_correlation(data_original, data_embedding)}"
    )

    print(
        f"DistCorr level 2: {dcor.distance_correlation(pdist(data_original, metric='cityblock'), pdist(data_embedding, metric='cityblock'))}"
    )

    # quality.qnx_crm
    print(f"Making Qnx_crm: {quality._return_qnx_crm(data_original, data_embedding,Qmatrix)}")
    # quality.rnx_crm
    print(f"Making Rnx_crm: {quality._return_rnx_crm(data_original, data_embedding,Qmatrix)}")
    # quality.rnx_auc_crm
    print(f"Making Qnx_auc_crm: {quality._return_rnx_auc_crm(data_original, data_embedding)}")

Making Qmatrix


computing coranking matrix: 100%|██████████| 500/500 [00:00<00:00, 4868.22it/s]


Qmatrix: [[17 10 17 10 12  6  4  6  9  6]
 [ 7 13  7  8 12  9  7  7  3  6]
 [17  6  7  7  8  5  7  4  6  6]
 [ 7  3  4  7  8  5 10  5  9  1]
 [14 10  5  3  3  3  4  3 11  4]
 [ 7  6  6  7  9  3  3  3  8  1]
 [14  4  5  3  8  6  4  4  5  4]
 [ 4  8  4  3  4  2  4  5  2  8]
 [ 7  3  7  5  6  7  4  1  3  1]
 [ 7  6  3  9  7  6  3  6  2  2]]
Making scores
Qnx: 0.40481042406097334
Qtrust: 0.7682598822143694
Qcont: 0.801123015312131
Qlcmc: 0.3044088176352705
QnMRRE: 0.24269459504973911
QvMRRE: 0.1930503246904574
DistCorr: [0.20843302179522483, 0.08650153185655748, 0.07441534364581859, 0.08521521181977286, 0.23849636418538245]
DistCorr level 1: 0.7415913292241197
DistCorr level 2: 0.41982542344142426
Making Qnx_crm: 0.30440881763527056
Making Rnx_crm: 0.22677587317492126


computing coranking matrix: 100%|██████████| 500/500 [00:00<?, ?it/s]


Making Qnx_auc_crm: 0.16568902948141306


In [4]:
from sklearn.preprocessing import StandardScaler
from hembedder.utils.quality_metrics import compute_coranking_matrix
from hembedder.utils import quality_metrics


def get_scores(embedded_info): 
    #Get performance metrics for each subsampled embedder
    evaluate_n_n_default = quality_metrics.CDEmbeddingPerformance(dcor_level=2, n_neighbours=15, metric='manhattan')
    evaluate = quality_metrics.CDEmbeddingPerformance(dcor_level=1,metric='manhattan')

    evaluators = {'Random_triplets' : evaluate.random_triplet_eval,
            'dcor_1': evaluate._return_distance_correlation,
            'neighbor_kept_50' : evaluate.neighbor_kept_ratio_eval,
            'neighbor_kept' : evaluate_n_n_default.neighbor_kept_ratio_eval,
            'dcor_2': evaluate_n_n_default._return_distance_correlation,
            'Trustworthiness': evaluate._return_trustworthiness,
            'LCMC_Q_matrix': evaluate._return_LCMC,
            'Trustworthiness_Q_matrix': evaluate._return_Qtrustworthiness,
            'Continuit_Q_matrix':  evaluate._return_Qcontinuity,
            'nMRRE_Q_matrix' : evaluate._return_nMRRE,
            'vMRRE_Q_matrix' : evaluate._return_vMRRE,
            'Qnx_crm_Q_matrix': evaluate._return_qnx_crm,
            'Rnx_crm_Q_matrix': evaluate._return_rnx_crm,
            'Qnx_auc_crm_Q_matrix': evaluate._return_rnx_auc_crm}
    embedded_info.update({'evaluators': evaluators})
    scores = metrics_scores_iter(**embedded_info)
    return scores

def get_embedded_data(X,parameter:dict, embedder,num_iter,subsampling):
    #Get embedded data from original data by subsampling
    X = np.asarray(X, dtype=np.float32)
    embedded_data = []
    times = []
    for iter in range(num_iter):
        start = time.time()
        sub = numpy_sampling(X, subsampling)
        # Evaluate randomly selected hyperparameters
        CD_scaled = sub.copy()
    
        scaler = StandardScaler()
        CD_scaled = scaler.fit_transform(sub)
        # Create a dictionary for later reference in multi-thread
        indexes_metrics= subsampling_return_indexes(sub, 5000)
        emb_dict = {"original" : sub[indexes_metrics],
                    "embedded" : embedder(**parameter).fit_transform(CD_scaled).astype(np.float32)[indexes_metrics]}
        embedded_data.append(emb_dict)
        #times.append(time.time()-start)
    return embedded_data

def get_embedded_without_embedding(X, X_embedded,num_iter,subsampling):
    #Get embedded data from original data by subsampling
    rng = np.random.default_rng(seed=42)
    X = np.asarray(X, dtype=np.float16)
    X_embedded = np.asarray(X_embedded,dtype=np.float16)
    embedded_data = []
    for iter in range(num_iter):
        sample_index = rng.choice(np.arange(X.shape[0]), size=subsampling, replace=False)
        # Create a dictionary for later reference in multi-thread
        emb_dict = {"x" : X[sample_index],
                    "output" : X_embedded[sample_index],
                    "Q":compute_coranking_matrix(X[sample_index],\
                    X_embedded[sample_index], leave = False).astype(np.int32)}
        embedded_data.append(emb_dict)
        
    return embedded_data

def metrics_scores_iter(
    x: np.array,
    output: np.array,
    evaluators: dict,
    **args
):
    """Calculates scores for embedder using different metrics (evaluators).

    Parameters
    ---------
    x: np.array
        original unemedded data
    output: np.array
        output array from the embedder for evaluation
    evaluators: dict
        further arguments to include the metrics (as function statement in a dict)
        if the functions take x and output as arguments.

    Returns
    ---------
    results: dict
        dictonary of metrics and their calculated scores.
    """
    results = {}
    for name, metric in evaluators.items():
        if("Q" in args.keys() and "Q_matrix" in name):
            results.update({name: metric(x, output,args["Q"])})
        else:
            results.update({name: metric(x, output)})
    return results

import queue
def get_embedded_without_embedding_queue(X, X_embedded,num_iter,subsampling):
    #Get embedded data from original data by subsampling
    rng = np.random.default_rng(seed=42)
    X = np.asarray(X, dtype=np.float16)
    X_embedded = np.asarray(X_embedded,dtype=np.float16)
    embedded_data = queue.Queue()
    for iter in range(num_iter):
        sample_index = rng.choice(np.arange(X.shape[0]), size=subsampling, replace=False)
        print(len(sample_index))
        # Create a dictionary for later reference in multi-thread
        emb_dict = {"original" : X[sample_index],
                    "embedded" : X_embedded[sample_index]}
        embedded_data.put(emb_dict)
        
    return embedded_data

def numpy_sampling(X, subsampling):  
    n_data = len(X) 
    idx = np.arange(n_data) 
    np.random.shuffle(idx) 
    return X[idx[: subsampling],:] 

def subsampling_return_indexes(X, subsampling):
    rand = np.random.default_rng()
    n_data = len(X) 
    subsampling = min(n_data, subsampling) 
    return  rand.choice(np.arange(n_data), size=subsampling, replace=False)

In [16]:
parameter = {"n_components":30}
#embedded_data_queue = get_embedded_without_embedding_queue(cell_dyn,dm6,10,10000,evaluators)
embedded_data = get_embedded_without_embedding(cell_dyn,dm6,10,1000)

In [8]:
#alculating performance for the embedded data using thread pool
from multiprocessing.dummy import Pool as ThreadPool
pool = ThreadPool(10)
scores=pool.map(get_scores, (x for x in embedded_data))
pool.close()
pool.join()

In [9]:
scores

[{'Random_triplets': 0.6980000000000001,
  'dcor_1': 0.857933315864784,
  'neighbor_kept_50': 0.36995999999999996,
  'neighbor_kept': 0.271,
  'dcor_2': 0.5484585475678636,
  'Trustworthiness': 0.8625359870200109,
  'LCMC_Q_matrix': 0.3703303303303303,
  'Trustworthiness_Q_matrix': 0.8707269010275819,
  'Continuit_Q_matrix': 0.8497877122769066,
  'nMRRE_Q_matrix': 0.11064360266384653,
  'vMRRE_Q_matrix': 0.13012838020526293,
  'Qnx_crm_Q_matrix': 0.3703303303303303,
  'Rnx_crm_Q_matrix': 0.33711990471484143,
  'Qnx_auc_crm_Q_matrix': 0.26562376230498014},
 {'Random_triplets': 0.6948000000000001,
  'dcor_1': 0.8436501322764045,
  'neighbor_kept_50': 0.35975999999999997,
  'neighbor_kept': 0.27353333333333335,
  'dcor_2': 0.5817533930486691,
  'Trustworthiness': 0.8567495294753922,
  'LCMC_Q_matrix': 0.3601201201201201,
  'Trustworthiness_Q_matrix': 0.860308577609519,
  'Continuit_Q_matrix': 0.8400186695511094,
  'nMRRE_Q_matrix': 0.12122244162660925,
  'vMRRE_Q_matrix': 0.13851077938053

In [7]:
results = Parallel(n_jobs=10,verbose=False,pre_dispatch='1.5*n_jobs')(delayed(get_scores)((embedded_data.get())) for _ in range(10))

AttributeError: 'list' object has no attribute 'get'

In [163]:

#sample_index = cell_dyn.sample(1000).index
print(sample_index[0:10])
sampled_cd = np.asarray(cell_dyn.loc[sample_index], dtype=np.float32)
sampled_emb = np.asarray(dm6.loc[sample_index], dtype=np.float32)
evaluate = CDEmbeddingPerformance(knn_dist='jaccard',n_neighbours=50,dcor_level=1,metric='manhattan')
Qmatrix = evaluate._get_coranking_matrix(
    X_org=data_original, X_emb=data_embedding, backend="numba"
)
print("Neigbor kept:",evaluate.neighbor_kept_ratio_eval(sampled_cd,sampled_emb))
print("Random triplet:",evaluate.random_triplet_eval(sampled_cd,sampled_emb))
print("Data correlation:",evaluate._return_distance_correlation(sampled_cd,sampled_emb))
print('LCMC_Q_matrix', evaluate._return_LCMC(sampled_cd,sampled_emb,Qmatrix))
print('Trustworthiness_Q_matrix', evaluate._return_Qtrustworthiness(sampled_cd,sampled_emb,Qmatrix))
print('Continuit_Q_matrix',  evaluate._return_Qcontinuity(sampled_cd,sampled_emb,Qmatrix))
print('nMRRE_Q_matrix' , evaluate._return_nMRRE(sampled_cd,sampled_emb,Qmatrix))
print('vMRRE_Q_matrix' , evaluate._return_vMRRE(sampled_cd,sampled_emb,Qmatrix))
print('Qnx_crm_Q_matrix', evaluate._return_qnx_crm(sampled_cd,sampled_emb,Qmatrix))
print('Rnx_crm_Q_matrix', evaluate._return_rnx_crm(sampled_cd,sampled_emb,Qmatrix))
print('Qnx_auc_crm_Q_matrix', evaluate._return_rnx_auc_crm(sampled_cd,sampled_emb,Qmatrix))

Int64Index([ 245790, 1437437, 2533969, 2574939, 1923116, 2057313,  655207,
            1434222, 2756490, 2481570],
           dtype='int64')


computing coranking matrix: 100%|██████████| 1000/1000 [00:00<00:00, 71620.37it/s]


Neigbor kept: 0.35494
Random triplet: 0.6906
Data correlation: 0.8550623904104924
LCMC_Q_matrix 0.3531131131131131
Trustworthiness_Q_matrix 0.8380783775013511
Continuit_Q_matrix 0.8650885884261764
nMRRE_Q_matrix 0.13998568056217425
vMRRE_Q_matrix 0.11631446431824963
Qnx_crm_Q_matrix 0.3531131131131131
Rnx_crm_Q_matrix 0.3189946064207667
Qnx_auc_crm_Q_matrix 0.26764033659620323


In [17]:
sample_index = cell_dyn.sample(40000).index
sampled_cd = np.asarray(cell_dyn.loc[sample_index], dtype=np.float32)
indexes_metrics= subsampling_return_indexes(sampled_cd, 1000)


In [18]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import trimap
from sklearn.decomposition import PCA

embedder_6 = umap.UMAP(n_components=6,n_neighbors=15,metric = 'manhattan', output_metric= 'manhattan',
                        n_jobs=4, n_epochs=500, init = 'spectral',dens_lambda=0.1, dens_frac=0.1, 
                        repulsion_strength=1.1, negative_sample_rate=10, random_state=None, min_dist=0.0)

embedder_15 = umap.UMAP(n_components=15,n_neighbors=15,metric = 'manhattan', output_metric= 'manhattan',
                        n_jobs=4, n_epochs=500, init = 'spectral',dens_lambda=0.1, dens_frac=0.1, 
                        repulsion_strength=1.1, negative_sample_rate=10, random_state=42, min_dist=0.0)

embedder_30 = umap.UMAP(n_components=30,n_neighbors=15,metric = 'manhattan', output_metric= 'manhattan',
                        n_jobs=4, n_epochs=500, init = 'spectral',dens_lambda=0.1, dens_frac=0.1, 
                        repulsion_strength=1.1, negative_sample_rate=10, random_state=42, min_dist=0.0)

embedder_60 = umap.UMAP(n_components=60,n_neighbors=15,metric = 'manhattan', output_metric= 'manhattan',
                        n_jobs=4, n_epochs=500, init = 'spectral',dens_lambda=0.1, dens_frac=0.1, 
                        repulsion_strength=1.1, negative_sample_rate=10, random_state=42, min_dist=0.0)

trimap_model_6 = trimap.TRIMAP(n_dims=6,n_inliers=100)
trimap_model_15 = trimap.TRIMAP(n_dims=15,n_inliers=100)
trimap_model_30 = trimap.TRIMAP(n_dims=30,n_inliers=100)
trimap_model_60 = trimap.TRIMAP(n_dims=60,n_inliers=100)


pca_model_6 = PCA(n_components=6,whiten=False)
pca_model_15 = PCA(n_components=15,whiten=False)
pca_model_30 = PCA(n_components=30,whiten=False)
pca_model_60 = PCA(n_components=60,whiten=False)




In [19]:
from scipy.signal import resample
le_pipe = Pipeline([('scaler', StandardScaler()),
                    ('embedder', embedder_6)])

le_pipe.fit(sampled_cd)
evaluate = CDEmbeddingPerformance(knn_dist='jaccard',n_neighbours=50,dcor_level=1,metric='manhattan')
eval_sample = sampled_cd[indexes_metrics].astype(np.float32)
eval_emb = le_pipe.named_steps['embedder'].embedding_[indexes_metrics].astype(np.float32)
print(le_pipe)
Qmatrix = quality._get_coranking_matrix(
    X_org=eval_sample, X_emb=eval_emb, backend="numba"
)
print(f"component number:{eval_emb.shape[1]}")
print("Neigbor kept:",evaluate.neighbor_kept_ratio_eval(eval_sample,eval_emb))
print("Random triplet:",evaluate.random_triplet_eval(eval_sample,eval_emb))
print("Data correlation:",evaluate._return_distance_correlation(eval_sample,eval_emb))
print('LCMC_Q_matrix', evaluate._return_LCMC(eval_sample,eval_emb,Qmatrix))
print('Trustworthiness_Q_matrix', evaluate._return_Qtrustworthiness(eval_sample,eval_emb,Qmatrix))
print('Continuit_Q_matrix',  evaluate._return_Qcontinuity(eval_sample,eval_emb,Qmatrix))
print('nMRRE_Q_matrix' , evaluate._return_nMRRE(eval_sample,eval_emb,Qmatrix))
print('vMRRE_Q_matrix' , evaluate._return_vMRRE(eval_sample,eval_emb,Qmatrix))
print('Qnx_crm_Q_matrix', evaluate._return_qnx_crm(eval_sample,eval_emb,Qmatrix))
print('Rnx_crm_Q_matrix', evaluate._return_rnx_crm(eval_sample,eval_emb,Qmatrix))
print('Qnx_auc_crm_Q_matrix', evaluate._return_rnx_auc_crm(eval_sample,eval_emb,Qmatrix))




Pipeline(steps=[('scaler', StandardScaler()),
                ('embedder',
                 UMAP(dens_frac=0.1, dens_lambda=0.1, metric='manhattan', min_dist=0.0, n_components=6, n_epochs=500, n_jobs=4, negative_sample_rate=10, output_metric='manhattan', repulsion_strength=1.1, tqdm_kwds={'bar_format': '{desc}: {percentage:3.0f}%| {bar} {n_fmt}/{total_fmt} [{elapsed}]', 'desc': 'Epochs completed', 'disable': True}))])


component number:6
Neigbor kept: 0.31895999999999997
Random triplet: 0.7316
Data correlation: 0.8314121390611661
LCMC_Q_matrix 0.3192792792792793
Trustworthiness_Q_matrix 0.8326470740941063
Continuit_Q_matrix 0.8740711736073558
nMRRE_Q_matrix 0.14994091839251278
vMRRE_Q_matrix 0.11232715034163432
Qnx_crm_Q_matrix 0.3192792792792793
Rnx_crm_Q_matrix 0.28337628768008516
Qnx_auc_crm_Q_matrix 0.26580794088317417


In [20]:
le_pipe = Pipeline([('scaler', StandardScaler()),
                    ('embedder', embedder_15)])

le_pipe.fit(sampled_cd)
evaluate = CDEmbeddingPerformance(knn_dist='jaccard',n_neighbours=15,dcor_level=1,metric='manhattan')
eval_sample = sampled_cd[indexes_metrics]
eval_emb = le_pipe.named_steps['embedder'].embedding_[indexes_metrics]
print(le_pipe)
Qmatrix = quality._get_coranking_matrix(
    X_org=eval_sample, X_emb=eval_emb, backend="numba"
)
print(f"component number:{eval_emb.shape[1]}")
print("Neigbor kept:",evaluate.neighbor_kept_ratio_eval(eval_sample,eval_emb))
print("Random triplet:",evaluate.random_triplet_eval(eval_sample,eval_emb))
print("Data correlation:",evaluate._return_distance_correlation(eval_sample,eval_emb))
print('LCMC_Q_matrix', evaluate._return_LCMC(eval_sample,eval_emb,Qmatrix))
print('Trustworthiness_Q_matrix', evaluate._return_Qtrustworthiness(eval_sample,eval_emb,Qmatrix))
print('Continuit_Q_matrix',  evaluate._return_Qcontinuity(eval_sample,eval_emb,Qmatrix))
print('nMRRE_Q_matrix' , evaluate._return_nMRRE(eval_sample,eval_emb,Qmatrix))
print('vMRRE_Q_matrix' , evaluate._return_vMRRE(eval_sample,eval_emb,Qmatrix))
print('Qnx_crm_Q_matrix', evaluate._return_qnx_crm(eval_sample,eval_emb,Qmatrix))
print('Rnx_crm_Q_matrix', evaluate._return_rnx_crm(eval_sample,eval_emb,Qmatrix))
print('Qnx_auc_crm_Q_matrix', evaluate._return_rnx_auc_crm(eval_sample,eval_emb,Qmatrix))

Pipeline(steps=[('scaler', StandardScaler()),
                ('embedder',
                 UMAP(dens_frac=0.1, dens_lambda=0.1, metric='manhattan', min_dist=0.0, n_components=15, n_epochs=500, n_jobs=4, negative_sample_rate=10, output_metric='manhattan', random_state=42, repulsion_strength=1.1, tqdm_kwds={'bar_format': '{desc}: {percentage:3.0f}%| {bar} {n_fmt}/{total_fmt} [{elapsed}]', 'desc': 'Epochs completed', 'disable': True}))])


component number:15
Neigbor kept: 0.21653333333333333
Random triplet: 0.7422
Data correlation: 0.837293519493167
LCMC_Q_matrix 0.2167500834167501
Trustworthiness_Q_matrix 0.8611037871033774
Continuit_Q_matrix 0.8992487205731831
nMRRE_Q_matrix 0.1304529010282032
vMRRE_Q_matrix 0.09659567956241343
Qnx_crm_Q_matrix 0.2167500834167501
Rnx_crm_Q_matrix 0.20479815183104436
Qnx_auc_crm_Q_matrix 0.26176613400052695


In [21]:
le_pipe = Pipeline([('scaler', StandardScaler()),
                    ('embedder', embedder_30)])

le_pipe.fit(sampled_cd)
evaluate = CDEmbeddingPerformance(knn_dist='jaccard',n_neighbours=15,dcor_level=1,metric='manhattan')
eval_sample = sampled_cd[indexes_metrics]
eval_emb = le_pipe.named_steps['embedder'].embedding_[indexes_metrics]
print(le_pipe)
Qmatrix = quality._get_coranking_matrix(
    X_org=eval_sample, X_emb=eval_emb, backend="numba"
)
print(f"component number:{eval_emb.shape[1]}")
print("Neigbor kept:",evaluate.neighbor_kept_ratio_eval(eval_sample,eval_emb))
print("Random triplet:",evaluate.random_triplet_eval(eval_sample,eval_emb))
print("Data correlation:",evaluate._return_distance_correlation(eval_sample,eval_emb))
print('LCMC_Q_matrix', evaluate._return_LCMC(eval_sample,eval_emb,Qmatrix))
print('Trustworthiness_Q_matrix', evaluate._return_Qtrustworthiness(eval_sample,eval_emb,Qmatrix))
print('Continuit_Q_matrix',  evaluate._return_Qcontinuity(eval_sample,eval_emb,Qmatrix))
print('nMRRE_Q_matrix' , evaluate._return_nMRRE(eval_sample,eval_emb,Qmatrix))
print('vMRRE_Q_matrix' , evaluate._return_vMRRE(eval_sample,eval_emb,Qmatrix))
print('Qnx_crm_Q_matrix', evaluate._return_qnx_crm(eval_sample,eval_emb,Qmatrix))
print('Rnx_crm_Q_matrix', evaluate._return_rnx_crm(eval_sample,eval_emb,Qmatrix))
print('Qnx_auc_crm_Q_matrix', evaluate._return_rnx_auc_crm(eval_sample,eval_emb,Qmatrix))

Pipeline(steps=[('scaler', StandardScaler()),
                ('embedder',
                 UMAP(dens_frac=0.1, dens_lambda=0.1, metric='manhattan', min_dist=0.0, n_components=30, n_epochs=500, n_jobs=4, negative_sample_rate=10, output_metric='manhattan', random_state=42, repulsion_strength=1.1, tqdm_kwds={'bar_format': '{desc}: {percentage:3.0f}%| {bar} {n_fmt}/{total_fmt} [{elapsed}]', 'desc': 'Epochs completed', 'disable': True}))])


component number:30
Neigbor kept: 0.21293333333333334
Random triplet: 0.7478
Data correlation: 0.8416469400952796
LCMC_Q_matrix 0.2131464798131465
Trustworthiness_Q_matrix 0.8558676902081201
Continuit_Q_matrix 0.8976244285226884
nMRRE_Q_matrix 0.13892264410215557
vMRRE_Q_matrix 0.09812893951338567
Qnx_crm_Q_matrix 0.21314647981314647
Rnx_crm_Q_matrix 0.20113955936268582
Qnx_auc_crm_Q_matrix 0.2574650013857491


In [22]:
le_pipe = Pipeline([('scaler', StandardScaler()),
                    ('embedder', embedder_60)])
le_pipe.fit(sampled_cd)
evaluate = CDEmbeddingPerformance(knn_dist='jaccard',n_neighbours=15,dcor_level=1,metric='manhattan')
eval_sample = sampled_cd[indexes_metrics]
eval_emb = le_pipe.named_steps['embedder'].embedding_[indexes_metrics]
print(le_pipe)
Qmatrix = quality._get_coranking_matrix(
    X_org=eval_sample, X_emb=eval_emb, backend="numba"
)
print(f"component number:{eval_emb.shape[1]}")
print("Neigbor kept:",evaluate.neighbor_kept_ratio_eval(eval_sample,eval_emb))
print("Random triplet:",evaluate.random_triplet_eval(eval_sample,eval_emb))
print("Data correlation:",evaluate._return_distance_correlation(eval_sample,eval_emb))
print('LCMC_Q_matrix', evaluate._return_LCMC(eval_sample,eval_emb,Qmatrix))
print('Trustworthiness_Q_matrix', evaluate._return_Qtrustworthiness(eval_sample,eval_emb,Qmatrix))
print('Continuit_Q_matrix',  evaluate._return_Qcontinuity(eval_sample,eval_emb,Qmatrix))
print('nMRRE_Q_matrix' , evaluate._return_nMRRE(eval_sample,eval_emb,Qmatrix))
print('vMRRE_Q_matrix' , evaluate._return_vMRRE(eval_sample,eval_emb,Qmatrix))
print('Qnx_crm_Q_matrix', evaluate._return_qnx_crm(eval_sample,eval_emb,Qmatrix))
print('Rnx_crm_Q_matrix', evaluate._return_rnx_crm(eval_sample,eval_emb,Qmatrix))
print('Qnx_auc_crm_Q_matrix', evaluate._return_rnx_auc_crm(eval_sample,eval_emb,Qmatrix))

Pipeline(steps=[('scaler', StandardScaler()),
                ('embedder',
                 UMAP(dens_frac=0.1, dens_lambda=0.1, metric='manhattan', min_dist=0.0, n_components=60, n_epochs=500, n_jobs=4, negative_sample_rate=10, output_metric='manhattan', random_state=42, repulsion_strength=1.1, tqdm_kwds={'bar_format': '{desc}: {percentage:3.0f}%| {bar} {n_fmt}/{total_fmt} [{elapsed}]', 'desc': 'Epochs completed', 'disable': True}))])


component number:60
Neigbor kept: 0.20633333333333334
Random triplet: 0.7482
Data correlation: 0.8400091763938826
LCMC_Q_matrix 0.20653987320653988
Trustworthiness_Q_matrix 0.8500644148754687
Continuit_Q_matrix 0.894979324462641
nMRRE_Q_matrix 0.14667538268975117
vMRRE_Q_matrix 0.10104495947409989
Qnx_crm_Q_matrix 0.20653987320653988
Rnx_crm_Q_matrix 0.19443213983736196
Qnx_auc_crm_Q_matrix 0.2490579763099267


In [23]:
scaled = StandardScaler().fit_transform(sampled_cd)
embedded = trimap_model_6.fit_transform(scaled)

eval_sample = sampled_cd[indexes_metrics]
eval_emb = embedded[indexes_metrics]
print(trimap_model_6)
Qmatrix = quality._get_coranking_matrix(
    X_org=eval_sample, X_emb=eval_emb, backend="numba"
)
print(f"component number:{eval_emb.shape[1]}")
print("Neigbor kept:",evaluate.neighbor_kept_ratio_eval(eval_sample,eval_emb))
print("Random triplet:",evaluate.random_triplet_eval(eval_sample,eval_emb))
print("Data correlation:",evaluate._return_distance_correlation(eval_sample,eval_emb))
print('LCMC_Q_matrix', evaluate._return_LCMC(eval_sample,eval_emb,Qmatrix))
print('Trustworthiness_Q_matrix', evaluate._return_Qtrustworthiness(eval_sample,eval_emb,Qmatrix))
print('Continuit_Q_matrix',  evaluate._return_Qcontinuity(eval_sample,eval_emb,Qmatrix))
print('nMRRE_Q_matrix' , evaluate._return_nMRRE(eval_sample,eval_emb,Qmatrix))
print('vMRRE_Q_matrix' , evaluate._return_vMRRE(eval_sample,eval_emb,Qmatrix))
print('Qnx_crm_Q_matrix', evaluate._return_qnx_crm(eval_sample,eval_emb,Qmatrix))
print('Rnx_crm_Q_matrix', evaluate._return_rnx_crm(eval_sample,eval_emb,Qmatrix))
print('Qnx_auc_crm_Q_matrix', evaluate._return_rnx_auc_crm(eval_sample,eval_emb,Qmatrix))

TRIMAP(n_dims=6, n_inliers=100,
       triplets=array([[    0, 15424,  3228],
       [    0, 15424, 21376],
       [    0, 15424,  6681],
       ...,
       [39999, 32039, 10343],
       [39999, 39772, 17810],
       [39999, 28809,  7514]]),
       weights=array([6.8382864, 6.5751486, 7.491781 , ..., 6.294773 , 6.2631865,
       6.334531 ], dtype=float32))


component number:6
Neigbor kept: 0.3280666666666667
Random triplet: 0.8068
Data correlation: 0.9025855110710718
LCMC_Q_matrix 0.32839506172839505
Trustworthiness_Q_matrix 0.9269045377004437
Continuit_Q_matrix 0.9419904469464346
nMRRE_Q_matrix 0.06863076261549153
vMRRE_Q_matrix 0.05383520007681549
Qnx_crm_Q_matrix 0.32839506172839505
Rnx_crm_Q_matrix 0.3181467666377806
Qnx_auc_crm_Q_matrix 0.36014600630198357


In [24]:
scaled = StandardScaler().fit_transform(sampled_cd)
embedded = trimap_model_15.fit_transform(scaled)

eval_sample = sampled_cd[indexes_metrics]
eval_emb = embedded[indexes_metrics]
print(trimap_model_15)
Qmatrix = quality._get_coranking_matrix(
    X_org=eval_sample, X_emb=eval_emb, backend="numba"
)
print(f"component number:{eval_emb.shape[1]}")
print("Neigbor kept:",evaluate.neighbor_kept_ratio_eval(eval_sample,eval_emb))
print("Random triplet:",evaluate.random_triplet_eval(eval_sample,eval_emb))
print("Data correlation:",evaluate._return_distance_correlation(eval_sample,eval_emb))
print('LCMC_Q_matrix', evaluate._return_LCMC(eval_sample,eval_emb,Qmatrix))
print('Trustworthiness_Q_matrix', evaluate._return_Qtrustworthiness(eval_sample,eval_emb,Qmatrix))
print('Continuit_Q_matrix',  evaluate._return_Qcontinuity(eval_sample,eval_emb,Qmatrix))
print('nMRRE_Q_matrix' , evaluate._return_nMRRE(eval_sample,eval_emb,Qmatrix))
print('vMRRE_Q_matrix' , evaluate._return_vMRRE(eval_sample,eval_emb,Qmatrix))
print('Qnx_crm_Q_matrix', evaluate._return_qnx_crm(eval_sample,eval_emb,Qmatrix))
print('Rnx_crm_Q_matrix', evaluate._return_rnx_crm(eval_sample,eval_emb,Qmatrix))
print('Qnx_auc_crm_Q_matrix', evaluate._return_rnx_auc_crm(eval_sample,eval_emb,Qmatrix))

TRIMAP(n_dims=15, n_inliers=100,
       triplets=array([[    0, 15424, 19766],
       [    0, 15424, 17953],
       [    0, 15424,  9526],
       ...,
       [39999,   301, 12924],
       [39999,  7374,  4620],
       [39999, 27268, 14299]]),
       weights=array([7.3979683, 8.351147 , 7.492214 , ..., 6.6569643, 6.6721582,
       6.783514 ], dtype=float32))


component number:15
Neigbor kept: 0.3283333333333333
Random triplet: 0.7976
Data correlation: 0.9023530926910625
LCMC_Q_matrix 0.328661995328662
Trustworthiness_Q_matrix 0.9268503582395088
Continuit_Q_matrix 0.9422067553735926
nMRRE_Q_matrix 0.06917706096215273
vMRRE_Q_matrix 0.053691940044720506
Qnx_crm_Q_matrix 0.32866199532866197
Rnx_crm_Q_matrix 0.3184177734872886
Qnx_auc_crm_Q_matrix 0.3576956214139882


In [25]:
scaled = StandardScaler().fit_transform(sampled_cd)
embedded = trimap_model_30.fit_transform(scaled)

eval_sample = sampled_cd[indexes_metrics]
eval_emb = embedded[indexes_metrics]
print(trimap_model_30)
Qmatrix = quality._get_coranking_matrix(
    X_org=eval_sample, X_emb=eval_emb, backend="numba"
)
print(f"component number:{eval_emb.shape[1]}")
print("Neigbor kept:",evaluate.neighbor_kept_ratio_eval(eval_sample,eval_emb))
print("Random triplet:",evaluate.random_triplet_eval(eval_sample,eval_emb))
print("Data correlation:",evaluate._return_distance_correlation(eval_sample,eval_emb))
print('LCMC_Q_matrix', evaluate._return_LCMC(eval_sample,eval_emb,Qmatrix))
print('Trustworthiness_Q_matrix', evaluate._return_Qtrustworthiness(eval_sample,eval_emb,Qmatrix))
print('Continuit_Q_matrix',  evaluate._return_Qcontinuity(eval_sample,eval_emb,Qmatrix))
print('nMRRE_Q_matrix' , evaluate._return_nMRRE(eval_sample,eval_emb,Qmatrix))
print('vMRRE_Q_matrix' , evaluate._return_vMRRE(eval_sample,eval_emb,Qmatrix))
print('Qnx_crm_Q_matrix', evaluate._return_qnx_crm(eval_sample,eval_emb,Qmatrix))
print('Rnx_crm_Q_matrix', evaluate._return_rnx_crm(eval_sample,eval_emb,Qmatrix))
print('Qnx_auc_crm_Q_matrix', evaluate._return_rnx_auc_crm(eval_sample,eval_emb,Qmatrix))

TRIMAP(n_dims=30, n_inliers=100,
       triplets=array([[    0, 15424, 15319],
       [    0, 15424,  2089],
       [    0, 15424, 30134],
       ...,
       [39999, 16866,  3980],
       [39999, 23163,  9417],
       [39999, 28677,  6399]]),
       weights=array([7.1482916, 7.5403175, 7.840534 , ..., 6.0202646, 6.2372847,
       6.06056  ], dtype=float32))


component number:30
Neigbor kept: 0.32426666666666665
Random triplet: 0.8008
Data correlation: 0.9020093814631172
LCMC_Q_matrix 0.32459125792459126
Trustworthiness_Q_matrix 0.9261061753667692
Continuit_Q_matrix 0.9418535653360628
nMRRE_Q_matrix 0.07046743587174875
vMRRE_Q_matrix 0.05397987417911751
Qnx_crm_Q_matrix 0.32459125792459126
Rnx_crm_Q_matrix 0.31428491903229105
Qnx_auc_crm_Q_matrix 0.357072148638206


In [26]:
scaled = StandardScaler().fit_transform(sampled_cd)
embedded = trimap_model_60.fit_transform(scaled)

eval_sample = sampled_cd[indexes_metrics]
eval_emb = embedded[indexes_metrics]
print(trimap_model_60)
Qmatrix = quality._get_coranking_matrix(
    X_org=eval_sample, X_emb=eval_emb, backend="numba"
)
print(f"component number:{eval_emb.shape[1]}")
print("Neigbor kept:",evaluate.neighbor_kept_ratio_eval(eval_sample,eval_emb))
print("Random triplet:",evaluate.random_triplet_eval(eval_sample,eval_emb))
print("Data correlation:",evaluate._return_distance_correlation(eval_sample,eval_emb))
print('LCMC_Q_matrix', evaluate._return_LCMC(eval_sample,eval_emb,Qmatrix))
print('Trustworthiness_Q_matrix', evaluate._return_Qtrustworthiness(eval_sample,eval_emb,Qmatrix))
print('Continuit_Q_matrix',  evaluate._return_Qcontinuity(eval_sample,eval_emb,Qmatrix))
print('nMRRE_Q_matrix' , evaluate._return_nMRRE(eval_sample,eval_emb,Qmatrix))
print('vMRRE_Q_matrix' , evaluate._return_vMRRE(eval_sample,eval_emb,Qmatrix))
print('Qnx_crm_Q_matrix', evaluate._return_qnx_crm(eval_sample,eval_emb,Qmatrix))
print('Rnx_crm_Q_matrix', evaluate._return_rnx_crm(eval_sample,eval_emb,Qmatrix))
print('Qnx_auc_crm_Q_matrix', evaluate._return_rnx_auc_crm(eval_sample,eval_emb,Qmatrix))

TRIMAP(n_dims=60, n_inliers=100,
       triplets=array([[    0, 15424, 13851],
       [    0, 15424, 22303],
       [    0, 15424, 17084],
       ...,
       [39999, 20240, 23637],
       [39999, 35311,  1902],
       [39999, 27358,   153]]),
       weights=array([6.918337 , 6.8965816, 7.0562963, ..., 5.9520035, 5.947644 ,
       5.9446588], dtype=float32))


component number:60
Neigbor kept: 0.32913333333333333
Random triplet: 0.7964
Data correlation: 0.9018334511685676
LCMC_Q_matrix 0.32946279612946283
Trustworthiness_Q_matrix 0.9263605595359947
Continuit_Q_matrix 0.941856021835551
nMRRE_Q_matrix 0.06961428244691169
vMRRE_Q_matrix 0.053983500154032554
Qnx_crm_Q_matrix 0.3294627961294628
Rnx_crm_Q_matrix 0.31923079403581267
Qnx_auc_crm_Q_matrix 0.3575831041152795


In [27]:
scaled = StandardScaler().fit_transform(sampled_cd)
embedded = pca_model_6.fit_transform(scaled)

eval_sample = sampled_cd[indexes_metrics]
eval_emb = embedded[indexes_metrics]
print(pca_model_6)
Qmatrix = quality._get_coranking_matrix(
    X_org=eval_sample, X_emb=eval_emb, backend="numba"
)
print(f"component number:{eval_emb.shape[1]}")
print("Neigbor kept:",evaluate.neighbor_kept_ratio_eval(eval_sample,eval_emb))
print("Random triplet:",evaluate.random_triplet_eval(eval_sample,eval_emb))
print("Data correlation:",evaluate._return_distance_correlation(eval_sample,eval_emb))
print('LCMC_Q_matrix', evaluate._return_LCMC(eval_sample,eval_emb,Qmatrix))
print('Trustworthiness_Q_matrix', evaluate._return_Qtrustworthiness(eval_sample,eval_emb,Qmatrix))
print('Continuit_Q_matrix',  evaluate._return_Qcontinuity(eval_sample,eval_emb,Qmatrix))
print('nMRRE_Q_matrix' , evaluate._return_nMRRE(eval_sample,eval_emb,Qmatrix))
print('vMRRE_Q_matrix' , evaluate._return_vMRRE(eval_sample,eval_emb,Qmatrix))
print('Qnx_crm_Q_matrix', evaluate._return_qnx_crm(eval_sample,eval_emb,Qmatrix))
print('Rnx_crm_Q_matrix', evaluate._return_rnx_crm(eval_sample,eval_emb,Qmatrix))
print('Qnx_auc_crm_Q_matrix', evaluate._return_rnx_auc_crm(eval_sample,eval_emb,Qmatrix))

PCA(n_components=6)


component number:6
Neigbor kept: 0.38339999999999996
Random triplet: 0.8762000000000001
Data correlation: 0.9703182949450857
LCMC_Q_matrix 0.3837837837837838
Trustworthiness_Q_matrix 0.9377617877857386
Continuit_Q_matrix 0.9748347321733197
nMRRE_Q_matrix 0.06385704992885587
vMRRE_Q_matrix 0.025297625566066094
Qnx_crm_Q_matrix 0.3837837837837838
Rnx_crm_Q_matrix 0.37438068791069806
Qnx_auc_crm_Q_matrix 0.44376689332919744


In [28]:
scaled = StandardScaler().fit_transform(sampled_cd)
embedded = pca_model_15.fit_transform(scaled)

eval_sample = sampled_cd[indexes_metrics]
eval_emb = embedded[indexes_metrics]
print(pca_model_15)
Qmatrix = quality._get_coranking_matrix(
    X_org=eval_sample, X_emb=eval_emb, backend="numba"
)
print(f"component number:{eval_emb.shape[1]}")
print("Neigbor kept:",evaluate.neighbor_kept_ratio_eval(eval_sample,eval_emb))
print("Random triplet:",evaluate.random_triplet_eval(eval_sample,eval_emb))
print("Data correlation:",evaluate._return_distance_correlation(eval_sample,eval_emb))
print('LCMC_Q_matrix', evaluate._return_LCMC(eval_sample,eval_emb,Qmatrix))
print('Trustworthiness_Q_matrix', evaluate._return_Qtrustworthiness(eval_sample,eval_emb,Qmatrix))
print('Continuit_Q_matrix',  evaluate._return_Qcontinuity(eval_sample,eval_emb,Qmatrix))
print('nMRRE_Q_matrix' , evaluate._return_nMRRE(eval_sample,eval_emb,Qmatrix))
print('vMRRE_Q_matrix' , evaluate._return_vMRRE(eval_sample,eval_emb,Qmatrix))
print('Qnx_crm_Q_matrix', evaluate._return_qnx_crm(eval_sample,eval_emb,Qmatrix))
print('Rnx_crm_Q_matrix', evaluate._return_rnx_crm(eval_sample,eval_emb,Qmatrix))
print('Qnx_auc_crm_Q_matrix', evaluate._return_rnx_auc_crm(eval_sample,eval_emb,Qmatrix))

PCA(n_components=15)


component number:15
Neigbor kept: 0.6606666666666666
Random triplet: 0.9410000000000001
Data correlation: 0.9929370326063357
LCMC_Q_matrix 0.6613279946613281
Trustworthiness_Q_matrix 0.9899703172978506
Continuit_Q_matrix 0.9952517911975435
nMRRE_Q_matrix 0.01117437946748195
vMRRE_Q_matrix 0.006406037268180344
Qnx_crm_Q_matrix 0.661327994661328
Rnx_crm_Q_matrix 0.6561600596866789
Qnx_auc_crm_Q_matrix 0.6775205455262224


In [29]:
scaled = StandardScaler().fit_transform(sampled_cd)
embedded = pca_model_30.fit_transform(scaled)

eval_sample = sampled_cd[indexes_metrics]
eval_emb = embedded[indexes_metrics]
print(pca_model_30)
Qmatrix = quality._get_coranking_matrix(
    X_org=eval_sample, X_emb=eval_emb, backend="numba"
)
print(f"component number:{eval_emb.shape[1]}")
print("Neigbor kept:",evaluate.neighbor_kept_ratio_eval(eval_sample,eval_emb))
print("Random triplet:",evaluate.random_triplet_eval(eval_sample,eval_emb))
print("Data correlation:",evaluate._return_distance_correlation(eval_sample,eval_emb))
print('LCMC_Q_matrix', evaluate._return_LCMC(eval_sample,eval_emb,Qmatrix))
print('Trustworthiness_Q_matrix', evaluate._return_Qtrustworthiness(eval_sample,eval_emb,Qmatrix))
print('Continuit_Q_matrix',  evaluate._return_Qcontinuity(eval_sample,eval_emb,Qmatrix))
print('nMRRE_Q_matrix' , evaluate._return_nMRRE(eval_sample,eval_emb,Qmatrix))
print('vMRRE_Q_matrix' , evaluate._return_vMRRE(eval_sample,eval_emb,Qmatrix))
print('Qnx_crm_Q_matrix', evaluate._return_qnx_crm(eval_sample,eval_emb,Qmatrix))
print('Rnx_crm_Q_matrix', evaluate._return_rnx_crm(eval_sample,eval_emb,Qmatrix))
print('Qnx_auc_crm_Q_matrix', evaluate._return_rnx_auc_crm(eval_sample,eval_emb,Qmatrix))

PCA(n_components=30)


component number:30
Neigbor kept: 0.8323333333333334
Random triplet: 0.976
Data correlation: 0.9984531319129384
LCMC_Q_matrix 0.8331664998331665
Trustworthiness_Q_matrix 0.9982116001364721
Continuit_Q_matrix 0.9990639372227909
nMRRE_Q_matrix 0.0030769554507460144
vMRRE_Q_matrix 0.002222257699399505
Qnx_crm_Q_matrix 0.8331664998331665
Rnx_crm_Q_matrix 0.8306207190574774
Qnx_auc_crm_Q_matrix 0.8392600051468195


In [30]:
scaled = StandardScaler().fit_transform(sampled_cd)
embedded = pca_model_60.fit_transform(scaled)

eval_sample = sampled_cd[indexes_metrics]
eval_emb = embedded[indexes_metrics]
print(pca_model_60)
Qmatrix = quality._get_coranking_matrix(
    X_org=eval_sample, X_emb=eval_emb, backend="numba"
)
print(f"component number:{eval_emb.shape[1]}")
print("Neigbor kept:",evaluate.neighbor_kept_ratio_eval(eval_sample,eval_emb))
print("Random triplet:",evaluate.random_triplet_eval(eval_sample,eval_emb))
print("Data correlation:",evaluate._return_distance_correlation(eval_sample,eval_emb))
print('LCMC_Q_matrix', evaluate._return_LCMC(eval_sample,eval_emb,Qmatrix))
print('Trustworthiness_Q_matrix', evaluate._return_Qtrustworthiness(eval_sample,eval_emb,Qmatrix))
print('Continuit_Q_matrix',  evaluate._return_Qcontinuity(eval_sample,eval_emb,Qmatrix))
print('nMRRE_Q_matrix' , evaluate._return_nMRRE(eval_sample,eval_emb,Qmatrix))
print('vMRRE_Q_matrix' , evaluate._return_vMRRE(eval_sample,eval_emb,Qmatrix))
print('Qnx_crm_Q_matrix', evaluate._return_qnx_crm(eval_sample,eval_emb,Qmatrix))
print('Rnx_crm_Q_matrix', evaluate._return_rnx_crm(eval_sample,eval_emb,Qmatrix))
print('Qnx_auc_crm_Q_matrix', evaluate._return_rnx_auc_crm(eval_sample,eval_emb,Qmatrix))

PCA(n_components=60)


component number:60
Neigbor kept: 0.9806666666666666
Random triplet: 0.9970000000000001
Data correlation: 0.9999504426390688
LCMC_Q_matrix 0.9816483149816484
Trustworthiness_Q_matrix 0.9999701125895599
Continuit_Q_matrix 0.9999701125895599
nMRRE_Q_matrix 0.00022338573497048252
vMRRE_Q_matrix 0.00022166357017603733
Qnx_crm_Q_matrix 0.9816483149816483
Rnx_crm_Q_matrix 0.9813682790963225
Qnx_auc_crm_Q_matrix 0.9842869560561471


In [ ]:
import cProfile, pstats, io
from pstats import SortKey
pr = cProfile.Profile()
pr.enable()
evaluate = CDEmbeddingPerformance(knn_dist='jaccard',n_neighbours=15)
evaluators = {'Trustworthiness': evaluate._return_trustworthiness,
                    'Knn overlap': evaluate._return_knn_overlap,
                    'Distance correlation': evaluate._return_distance_correlation,
                    'Random triplets' : evaluate.random_triplet_eval,
                    'neighbor kept ratio' : evaluate.neighbor_kept_ratio_eval}
final_results=score_subsampling(sampled_cd,sampled_embedded,evaluators,size=20000)
pr.disable()
s = io.StringIO()
sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats()
print(s.getvalue())

         1443 function calls in 217.289 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        4    0.000    0.000  217.288   54.322 c:\Users\cchumsae\Anaconda3\envs\CellDynCluster\lib\site-packages\IPython\core\interactiveshell.py:3362(run_code)
        4    0.000    0.000  217.288   54.322 {built-in method builtins.exec}
        1    0.000    0.000  217.288  217.288 C:\Users\cchumsae\AppData\Local\Temp\ipykernel_20596\3697458949.py:1(<cell line: 11>)
        1    0.000    0.000  217.288  217.288 C:\Users\cchumsae\AppData\Local\Temp\ipykernel_20596\865909864.py:295(score_subsampling)
        1    0.000    0.000  217.288  217.288 c:\Users\cchumsae\Anaconda3\envs\CellDynCluster\lib\site-packages\joblib\parallel.py:960(__call__)
        1    0.000    0.000  217.175  217.175 c:\Users\cchumsae\Anaconda3\envs\CellDynCluster\lib\site-packages\joblib\parallel.py:920(retrieve)
       10    0.000    0.000  217.175   21.717 c:\Use

In [6]:
cProfile.run("evaluate._return_trustworthiness(sampled_cd,sampled_embedded)")

         704 function calls (685 primitive calls) in 34.516 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(all)
        1    0.000    0.000   25.814   25.814 <__array_function__ internals>:2(argsort)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(atleast_2d)
        1    0.000    0.000    0.001    0.001 <__array_function__ internals>:2(concatenate)
        1    0.000    0.000    0.001    0.001 <__array_function__ internals>:2(copyto)
        1    0.000    0.000    0.002    0.002 <__array_function__ internals>:2(einsum)
        2    0.000    0.000    0.002    0.001 <__array_function__ internals>:2(fill_diagonal)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(reshape)
        5    0.000    0.000    0.004    0.001 <__array_function__ internals>:2(sum)
        1    0.000    0.000    0.0